# Introductory sheaves tutorial


Here we will run through a simple sheaf example using Micahel Robinson's [pysheaf](https://github.com/kb1dds/pysheaf) python package. The code is still under active development so we will demonstrate a simple example from Figure 3 of Blevins and Bassett 2020.

In [1]:
## Import python pysheaf package.
using PyCall
ps = pyimport("pysheaf")


┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1242


PyObject <module 'pysheaf' from '/opt/conda/lib/python3.7/site-packages/pysheaf-0.3.1-py3.7.egg/pysheaf/__init__.py'>

### Cell signaling example

In the cell signaling example from Figure 3a, our system is set up

(Cell 1) --------> (Cell 2) --------> (Cell 3)

and the effect across edges is

(Cell 1) --- 2x ---> (Cell 2) --- 0.5x ---> (Cell 3)

so that cell 2 signals at twice the rate of cell 1, and cell 3 signals at half the rate of cell 2.

We start to represent this system the nodes of our system with `AddCell`.


In [2]:
shf=ps.Sheaf()

shf.AddCell("Cell_1",ps.Cell("real")) # First argument is the name of the Cell, the second describes the stalk
shf.AddCell("Cell_2",ps.Cell("real"))
shf.AddCell("Cell_3",ps.Cell("real"))


At this point we have three nodes, one for each cell, and we link them together with restriction maps following the edges in our system.

In [3]:
# Defining functions for restriction maps
function mult_by_two(x)
    return 2.0*x
end

function mult_by_half(x)
    return 0.5*x
end


# Add these relations to the sheaf
shf.AddCoface("Cell_1","Cell_2",ps.Coface("real","real",mult_by_two))
shf.AddCoface("Cell_2","Cell_3",ps.Coface("real","real",mult_by_half))

Now in order to reproduce the figure, we can assign the rates for each cell:

| Cell Number   | Rate (number of molecules per time)        |
| ------------- |-------------------------------------------:|
| Cell 1        | 4                                          |
| Cell 2        | 8                                          |
| Cell 3        | 4                                          |


and we know these also satisfy our maps. We will add the data assignments with `SetDataAssignment`.

In [4]:
shf.GetCell("Cell_1").SetDataAssignment(ps.Assignment("real",4.0))
shf.GetCell("Cell_2").SetDataAssignment(ps.Assignment("real",8.0))
shf.GetCell("Cell_3").SetDataAssignment(ps.Assignment("real",4.0))


The next few lines of code check the consistency radius of the sheaf. For details see slide 16 of this [presentation](http://www.appliedcategorytheory.org/wp-content/uploads/2018/03/Michael-Robinson-Sheaf-Methods-for-Inference.pdf)

Briefly, if we pick one node and propagate that value along the sheaf following the restriction maps, we may get different values at each node than those assigned. The consistency radius is the maximum difference between the actual data assignment and an expected value via propagation, and sets a lower bound on the distance to the nearest global section [1](https://arxiv.org/abs/1603.01446).

In [5]:
shf.mPreventRedundantExtendedAssignments = false

shf.MaximallyExtendCell("Cell_1")   # Use the data on Cell 1 to decide what values at other nodes *should* be
shf.MaximallyExtendCell("Cell_2")
shf.MaximallyExtendCell("Cell_3")

consistency_radius = shf.ComputeConsistencyRadius()

print("The consistency radius is $consistency_radius")

The consistency radius is 0.0

### New observations

If we observe new data from the system, will the new data fit our sheaf?

In [6]:
# If we add drug A and then we observe the following rates:

shf.GetCell("Cell_1").SetDataAssignment(ps.Assignment("real",2.0))
shf.GetCell("Cell_2").SetDataAssignment(ps.Assignment("real",4.0))
shf.GetCell("Cell_3").SetDataAssignment(ps.Assignment("real",2.0))

# Check to see if the new data is consistent with our original sheaf

shf.mPreventRedundantExtendedAssignments = false

shf.MaximallyExtendCell("Cell_1")
shf.MaximallyExtendCell("Cell_2")
shf.MaximallyExtendCell("Cell_3")

consistency_radius_A = shf.ComputeConsistencyRadius()

print("The consistency radius is $consistency_radius_A")


The consistency radius is 0.0

In [7]:
# If we add drug B and then we observe the following rates:

shf.GetCell("Cell_1").SetDataAssignment(ps.Assignment("real",4.0))
shf.GetCell("Cell_2").SetDataAssignment(ps.Assignment("real",4.0))
shf.GetCell("Cell_3").SetDataAssignment(ps.Assignment("real",2.0))

# Check to see if the new data is consistent with our original sheaf

shf.mPreventRedundantExtendedAssignments = false

shf.MaximallyExtendCell("Cell_1")
shf.MaximallyExtendCell("Cell_2")
shf.MaximallyExtendCell("Cell_3")

consistency_radius_B = shf.ComputeConsistencyRadius()

print("The consistency radius is $consistency_radius_B")

The consistency radius is 4.0

Now we can see that our new data does not fit the original model!

| Cell Number   | Rate original       | Rate after perturbation 1|Rate after perturbation 2|
| ------------- |-------------------------------------------:|-------------------------------------------:|-------------------------------------------:|
| Cell 1        | 4                                          | 2| 4
| Cell 2        | 8                                          | 4 | 4 |
| Cell 3        | 4                                          | 2 | 2 |
|               |
| Consistency radius | 0 | 0 | 2|